In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "simulation"

## Prerequisites


### Example batch case

The following batch case is used as an example.


In [2]:
from models.case import BatchCase, Transaction

example_case = BatchCase(
    id="swap_usdt_usdc",
    description="Swap 100 USDT for USDC on Uniswap and transfer to Alice",
    steps=[
        Transaction(description="Approve USDT for Uniswap"),
        Transaction(description="Swap 100 USDT to USDC on Uniswap"),
        Transaction(description="Transfer USDC to Alice"),
    ],
)

### Contract addresses

Assume the following contract addresses are known.


In [3]:
# Contract addresses
USDT_ADDR = "0xdAC17F958D2ee523a2206206994597C13D831ec7"
USDC_ADDR = "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"
UNISWAP_V2_ROUTER_ADDR = "0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D"

# Dummy EOA
ALICE_ADDR = "0x436f795B64E23E6cE7792af4923A68AFD3967952"
BOB_ADDR = "0x8c575b178927fF9A70804B8b4F7622F7666bB360"

### Tools

We use web3.py to create the instances of the contracts.


In [4]:
import os
import json
from web3 import Web3

INFURA_API_KEY = os.getenv("INFURA_API_KEY")
web3 = Web3(Web3.HTTPProvider(f"https://mainnet.infura.io/v3/{INFURA_API_KEY}"))

with open("abi/erc20.json") as file:
    erc20_contract_json = json.load(file)

with open("abi/uniswap_v2_router.json") as file:
    uniswap_contract_json = json.load(file)

# Contract instances
usdc_contract = web3.eth.contract(address=USDC_ADDR, abi=erc20_contract_json)
usdt_contract = web3.eth.contract(address=USDT_ADDR, abi=erc20_contract_json)
uniswap_contract = web3.eth.contract(
    address=UNISWAP_V2_ROUTER_ADDR, abi=uniswap_contract_json
)

Example usage of the contract instances.


In [5]:
def get_balance(token_address: str, address: str):
    contract_instance = web3.eth.contract(
        address=token_address, abi=erc20_contract_json
    )
    encoded = contract_instance.encode_abi("balanceOf", args=[address])
    tx = {
        "to": token_address,
        "data": encoded,
    }

    try:
        # Perform the call
        result = web3.eth.call(tx)
        # Decode the result (balance) from bytes to integer
        decoded_result = int.from_bytes(result)
        # Convert to USDC (assuming 6 decimal places)
        usdc_balance = decoded_result / 10**6
        return usdc_balance
    except Exception as e:
        # Handle any errors that occur during the call
        print(f"An error occurred: {e}")


# Get balance of Bob
print("Bob's USDT balance:", get_balance(USDT_ADDR, BOB_ADDR))
print("Bob's USDC balance:", get_balance(USDC_ADDR, BOB_ADDR))

# Get balance of Alice
print("Alice's USDT balance:", get_balance(USDT_ADDR, ALICE_ADDR))
print("Alice's USDC balance:", get_balance(USDC_ADDR, ALICE_ADDR))

Bob's USDT balance: 0.0
Bob's USDC balance: 0.0
Alice's USDT balance: 4.0
Alice's USDC balance: 4.44435


## Simulate the transaction

Refer to the [Tenderly API documentation](https://docs.tenderly.co/node/rpc-reference/ethereum-mainnet/tenderly_simulateBundle) for more details.


In [6]:
import os
import requests
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field, validator


class TransactionParams(BaseModel):
    from_address: str = Field(description="The sender address")
    to_address: str = Field(description="The receiver address")
    data: str = Field(description="The data of the transaction")
    value: str = Field(description="The value of native token to send")

    @validator("from_address", "to_address")
    def check_address_format(cls, v):
        if not v.startswith("0x"):
            raise ValueError("Address must start with '0x'")
        if len(v) != 42:
            raise ValueError("Address must be 42 characters long")
        return v

    @validator("value")
    def check_value_format(cls, v):
        if not v.startswith("0x"):
            raise ValueError("Value must start with '0x'")
        return v


def simulate_transaction(transactions: List[TransactionParams]):
    TENDERLY_API_KEY = os.getenv("TENDERLY_API_KEY")
    url = f"https://mainnet.gateway.tenderly.co/{TENDERLY_API_KEY}"
    data = {
        "id": 0,
        "jsonrpc": "2.0",
        "method": "tenderly_simulateBundle",
        "params": [
            [
                {
                    "from": tx.from_address,
                    "to": tx.to_address,
                    "data": tx.data,
                    "value": tx.value,
                }
                for tx in transactions
            ],
            "latest",
        ],
    }
    response = requests.post(url, json=data).json()
    if "error" in response:
        raise Exception(f"Simulation failed: {response['error']['message']}")
    if "result" in response:
        return response["result"]

    raise Exception(f"Unexpected response: {response}")

In [7]:
def format_simulation_results(results: list[dict]):
    formatted_results = []
    for transaction in results:
        success = transaction["status"]
        formatted_logs = [
            {key: log[key] for key in log if key not in {"anonymous", "raw"}}
            for log in transaction["logs"]
        ]
        asset_changes = [
            {
                "asset_info": {
                    "contract_address": change["assetInfo"]["contractAddress"],
                    "symbol": change["assetInfo"]["symbol"],
                    "decimals": change["assetInfo"]["decimals"],
                },
                "type": change["type"],
                "from": change["from"],
                "to": change["to"],
                "raw_amount": change["rawAmount"],
            }
            for change in transaction.get("assetChanges", [])
        ]
        trace_list = transaction.get("trace", [])
        trace = trace_list[-1] if trace_list and not success else None

        formatted_transaction = {"success": success}
        if formatted_logs:
            formatted_transaction["logs"] = formatted_logs
        if asset_changes:
            formatted_transaction["asset_changes"] = asset_changes
        if trace:
            formatted_transaction["trace"] = trace
        formatted_results.append(formatted_transaction)
    return formatted_results

In [8]:
from utils.model_selector import get_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

llm = get_chat_model(temperature=0).model

system_prompt = """You will be provided with the results of a transaction simulation.
Extract the asset changes from the transaction. Do NOT modify the data. If the transaction failed, the asset changes will be empty."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)


class AssetChange(BaseModel):
    contract_address: str = Field(description="The contract address of the asset")
    name: str = Field(description="The name of the asset")
    symbol: str = Field(description="The symbol of the asset")
    token_decimals: int = Field(description="The decimals of the token.")
    raw_amount: str = Field(description="The raw amount in hex. Field 'rawAmount'")
    sender: str = Field(description="The sender of the transaction. Field 'from'")
    receiver: str = Field(description="The receiver of the transaction. Field 'to'")

    # @validator("contract_address")
    # def check_token_address_format(cls, v):
    #     if not v.startswith("0x"):
    #         raise ValueError("Token address must start with '0x'")
    #     if len(v) != 42:
    #         raise ValueError("Token address must be 42 characters long")
    #     return v


class SimulatedTransaction(BaseModel):
    asset_changes: List[AssetChange] = Field(
        description="List of assets modified during the transaction."
    )
    error: str = Field(
        description="Detailed error message explaining why the transaction failed, combining 'error' and 'errorReason' from the trace. This field is empty if the transaction succeeded."
    )


chain = prompt | llm.with_structured_output(SimulatedTransaction)

In [9]:
def pretty_print(result: SimulatedTransaction):
    """
    Print the result of the simulation generated by the LLM in a readable format.
    """
    print(f"The transaction was {'failed. 😕' if result.error else 'successful! 🎉'}")
    if result.error:
        print("-------------")
        print(f"Error: {result.error}")
    for asset_change in result.asset_changes:
        outgoing_sign = (
            "-" if asset_change.sender.lower() == ALICE_ADDR.lower() else "+"
        )
        amount = int(asset_change.raw_amount, 16)
        print("-------------")
        print(f"{asset_change.symbol.upper()} ({asset_change.contract_address})")
        print(f"Change: {outgoing_sign}{amount:,} ({asset_change.raw_amount})")

## Example - Successful


In [10]:
import time

example_tx = [
    TransactionParams(
        from_address=ALICE_ADDR,
        to_address=USDC_ADDR,
        data=usdc_contract.encode_abi(
            "approve", args=[UNISWAP_V2_ROUTER_ADDR, 1_000_000]
        ),
        value="0x0",
    ),
    TransactionParams(
        from_address=ALICE_ADDR,
        to_address=UNISWAP_V2_ROUTER_ADDR,
        data=uniswap_contract.encode_abi(
            "swapExactTokensForTokens",
            args=[
                1_000_000,
                0,
                [USDC_ADDR, USDT_ADDR],
                ALICE_ADDR,
                int(time.time()) + 500,
            ],
        ),
        value="0x0",
    ),
    TransactionParams(
        from_address=ALICE_ADDR,
        to_address=USDT_ADDR,
        data=usdc_contract.encode_abi("transfer", args=[BOB_ADDR, 4_000_001]),
        value="0x0",
    ),
]

tx_results = format_simulation_results(simulate_transaction(example_tx))
print(tx_results)

[{'success': True, 'logs': [{'name': 'Approval', 'inputs': [{'value': '0x436f795b64e23e6ce7792af4923a68afd3967952', 'type': 'address', 'name': 'owner', 'indexed': True}, {'value': '0x7a250d5630b4cf539739df2c5dacb4c659f2488d', 'type': 'address', 'name': 'spender', 'indexed': True}, {'value': '1000000', 'type': 'uint256', 'name': 'value', 'indexed': False}]}]}, {'success': True, 'logs': [{'name': 'Transfer', 'inputs': [{'value': '0x436f795b64e23e6ce7792af4923a68afd3967952', 'type': 'address', 'name': 'from', 'indexed': True}, {'value': '0x3041cbd36888becc7bbcbc0045e3b1f144466f5f', 'type': 'address', 'name': 'to', 'indexed': True}, {'value': '1000000', 'type': 'uint256', 'name': 'value', 'indexed': False}]}, {'name': 'Transfer', 'inputs': [{'value': '0x3041cbd36888becc7bbcbc0045e3b1f144466f5f', 'type': 'address', 'name': 'from', 'indexed': True}, {'value': '0x436f795b64e23e6ce7792af4923a68afd3967952', 'type': 'address', 'name': 'to', 'indexed': True}, {'value': '998343', 'type': 'uint256'

In [11]:
result = chain.invoke({"input": tx_results})
print(result.json())

{"asset_changes": [{"contract_address": "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48", "name": "usdc", "symbol": "usdc", "token_decimals": 6, "raw_amount": "0xf4240", "sender": "0x436f795b64e23e6ce7792af4923a68afd3967952", "receiver": "0x3041cbd36888becc7bbcbc0045e3b1f144466f5f"}, {"contract_address": "0xdac17f958d2ee523a2206206994597c13d831ec7", "name": "usdt", "symbol": "usdt", "token_decimals": 6, "raw_amount": "0xf3bc7", "sender": "0x3041cbd36888becc7bbcbc0045e3b1f144466f5f", "receiver": "0x436f795b64e23e6ce7792af4923a68afd3967952"}, {"contract_address": "0xdac17f958d2ee523a2206206994597c13d831ec7", "name": "usdt", "symbol": "usdt", "token_decimals": 6, "raw_amount": "0x3d0901", "sender": "0x436f795b64e23e6ce7792af4923a68afd3967952", "receiver": "0x8c575b178927ff9a70804b8b4f7622f7666bb360"}], "error": ""}


In [12]:
pretty_print(result)

The transaction was successful! 🎉
-------------
USDC (0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48)
Change: -1,000,000 (0xf4240)
-------------
USDT (0xdac17f958d2ee523a2206206994597c13d831ec7)
Change: +998,343 (0xf3bc7)
-------------
USDT (0xdac17f958d2ee523a2206206994597c13d831ec7)
Change: -4,000,001 (0x3d0901)


## Example - Failed


In [13]:
import time

example_tx = [
    TransactionParams(
        from_address=ALICE_ADDR,
        to_address=UNISWAP_V2_ROUTER_ADDR,
        data=uniswap_contract.encode_abi(
            "swapExactTokensForTokens",
            args=[
                1_000_000,
                0,
                [USDC_ADDR, USDT_ADDR],
                ALICE_ADDR,
                int(time.time()) + 500,
            ],
        ),
        value="0x0",
    ),
]

tx_results = format_simulation_results(simulate_transaction(example_tx))
print(tx_results)
print("-------------")
result = chain.invoke({"input": tx_results})
print(result.json())

[{'success': False, 'trace': {'type': 'DELEGATECALL', 'from': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'to': '0x43506849d7c04f9138d1a2050bbf3a0c054402dd', 'gas': '0x22a6c5d6', 'gasUsed': '0x2d35', 'error': 'execution reverted', 'errorReason': 'ERC20: transfer amount exceeds allowance', 'input': '0x23b872dd000000000000000000000000436f795b64e23e6ce7792af4923a68afd39679520000000000000000000000003041cbd36888becc7bbcbc0045e3b1f144466f5f00000000000000000000000000000000000000000000000000000000000f4240', 'decodedInput': [{'value': '0x436f795b64e23e6ce7792af4923a68afd3967952', 'type': 'address', 'name': 'from', 'indexed': False}, {'value': '0x3041cbd36888becc7bbcbc0045e3b1f144466f5f', 'type': 'address', 'name': 'to', 'indexed': False}, {'value': '1000000', 'type': 'uint256', 'name': 'value', 'indexed': False}], 'method': 'transferFrom', 'output': '0x', 'subtraces': 0, 'traceAddress': [1, 0]}}]
-------------
{"asset_changes": [], "error": "execution reverted: ERC20: transfer amount exceeds 

In [14]:
pretty_print(result)

The transaction was failed. 😕
-------------
Error: execution reverted: ERC20: transfer amount exceeds allowance
